In [1]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.CepsMy1Experiment import CepsMy1Experiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [3]:
import sqlite3
conn = sqlite3.connect("database/2024_12_15_20_44_49.db")
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,centroid_centroid_x,centroid_centroid_y,centroid_dist_min,centroid_dist_median,centroid_dist_mean,centroid_dist_max,centroid_dist_sd,centroid_dist_span,centroid_dist_coef_of_var,cluster_01pct_number_of_clusters,cluster_01pct_mean_distance_to_centroid,cluster_05pct_number_of_clusters,cluster_05pct_mean_distance_to_centroid,cluster_10pct_number_of_clusters,cluster_10pct_mean_distance_to_centroid,bounding_box_10_ratio_of_cities_outside_box,bounding_box_20_ratio_of_cities_outside_box,bounding_box_30_ratio_of_cities_outside_box,chull_area,chull_points_on_hull,distance_distances_shorter_mean_distance,distance_distinct_distances,distance_mode_frequency,distance_mode_quantity,distance_mode_mean,distance_mean_tour_length,distance_sum_of_lowest_edge_values,distance_min,distance_median,distance_mean,distance_max,distance_sd,distance_span,distance_coef_of_var,modes_number,mst_depth_min,mst_depth_median,mst_depth_mean,mst_depth_max,mst_depth_sd,mst_depth_span,mst_depth_coef_of_var,mst_dists_min,mst_dists_median,mst_dists_mean,mst_dists_max,mst_dists_sd,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var,num_nodes,cost_matrix_avg,cost_matrix_std,cost_matrix_skew,stdTime,mst_length,mst_length_avg,mst_length_std,mst_length_skew,mst_degree_avg,mst_degree_std,mst_degree_skew,mstTime,cluster_distance_avg,cluster_distance_std,cluster_distance_skew,clusterTime,tour_const_heu_avg,tour_const_std,tour_const_skew,ls_impov_per_step_avg,ls_impov_per_step_std,ls_impov_per_step_skew,ls_steps_2lm_avg,ls_steps_2lm_std,ls_steps_2lm_skew,ls_maxdist_avg,ls_maxdist_std,ls_maxdist_skew,ls_bestsol_avg,ls_bestsol_std,ls_bestsol_skew,ls_backbone_avg,ls_backbone_std,ls_backbone_skew,lpTime,bc_improv_per_cut_avg,bc_improv_per_cut_std,bc_improv_per_cut_skew,bc_upper_lower_ratio,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,0.001897,1.430042,1.491193,3.140406,0.927441,3.138509,0.621946,0.471871,0.487806,0.040004,0.369238,0.381506,0.706402,0.151182,0.666398,0.396278,23.0,0.024919,7.0,0.047305,5.0,0.096363,0.30250,0.45500,0.85500,0.739118,0.02250,0.458728,1.0,800.0,0.000002,0.0,818.771792,4.210875,0.0,0.533828,0.511093,1.350294,0.274833,1.350294,0.537736,4.0,1.0,5.0,13.24125,80.0,18.549748,79.0,1.400906,0.000219,0.008216,0.011043,0.339508,0.016342,0.339289,1.479878,0.000027,0.000219,0.005963,0.008050,0.070918,0.007696,0.070699,0.956124,800.0,510536.387909,0.536197,-38.723116,0.02,0.021553,0.021580,1.478438,0.079151,1.9975,0.341379,19.833117,0.02,0.363833,0.699597,14.795844,0.18,0.031919,0.000000e+00,0.000000e+00,0.002602,0.000824,-6.529526e+00,0.120875,0.006375,-1.050584,0.139375,2.478186e-02,1.798802e+00,20.848407,1.742264e-02,4.868199e+01,0.813835,0.279135,-0.884707,2.93,0.010508,4.320124,0.547672,0.010640,0.111111,0.500000,0.5,0.5,0.888889,0.89750,0.20625,0.22,380.733368,0.43
1,data/TSP/CEPS_benchmark/grid/04.tsp,0.001978,1.651404,1.641448,3.141593,0.934762,3.139615,0.569474,0.517514,0.488799,0.010197,0.405103,0.386497,0.717120,0.138593,0.706924,0.358587,1.0,0.384896,1.0,0.386497,1.0,0.386497,0.37375,0.66125,0.85125,0.984655,0.01875,0.510500,1.0,800.0,0.000002,0.0,840.219791,16.411909,0.0,0.516829,0.524481,1.381468,0.249095,1.381468,0.474936,2.0,1.0,5.0,13.65625,83.0,19.182819,82.0,1.404692,0.001058,0.024411,0.024615,0.067842,0.010731,0.066784,0.435948,0.000059,0.001058,0.018953,0.019898,0.067842,0.010276,0.066784,0.516433,800.0,524785.337480,0.473334,6.460961,0.02,0.046818,0.046877,0.435728,9.496167,1.9975,0.346841,6.204244,0.02,0.082693,0.098202,-1.880810,0.94,0.060078,0.000000e+00,0.000000e+00,0.005326,0.000000,-9.000000e-09,0.105000,0.000000,0.000000,0.150000,0.000000e+00,0.000000e+00,41.436466,6.030000e-07,-3.000000e-09,1.000000,0.000000,0.000000,0.91,0.001727,2.622374,0.410356,0.011900,0.250000,0.500000,0.5,0.5

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [3]:
import sqlite3
conn = sqlite3.connect("_archive/7/MY_CEPS1.db")
df_instances = pd.read_sql_query("SELECT * FROM instances WHERE angle_min IS NOT NULL", conn)
conn.close()

from src.database import db_connect, db_init
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
db_init(solver_class, instance_class, calculate_instance_features=True)
conn = db_connect()

In [4]:
df_instances.to_sql("instances", conn, if_exists="append", index=False)

238

In [5]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id,angle_min,angle_median,angle_mean,angle_max,angle_sd,angle_span,angle_coef_of_var,centroid_centroid_x,centroid_centroid_y,centroid_dist_min,centroid_dist_median,centroid_dist_mean,centroid_dist_max,centroid_dist_sd,centroid_dist_span,centroid_dist_coef_of_var,cluster_01pct_number_of_clusters,cluster_01pct_mean_distance_to_centroid,cluster_05pct_number_of_clusters,cluster_05pct_mean_distance_to_centroid,cluster_10pct_number_of_clusters,cluster_10pct_mean_distance_to_centroid,bounding_box_10_ratio_of_cities_outside_box,bounding_box_20_ratio_of_cities_outside_box,bounding_box_30_ratio_of_cities_outside_box,chull_area,chull_points_on_hull,distance_distances_shorter_mean_distance,distance_distinct_distances,distance_mode_frequency,distance_mode_quantity,distance_mode_mean,distance_mean_tour_length,distance_sum_of_lowest_edge_values,distance_min,distance_median,distance_mean,distance_max,distance_sd,distance_span,distance_coef_of_var,modes_number,mst_depth_min,mst_depth_median,mst_depth_mean,mst_depth_max,mst_depth_sd,mst_depth_span,mst_depth_coef_of_var,mst_dists_min,mst_dists_median,mst_dists_mean,mst_dists_max,mst_dists_sd,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var,num_nodes,cost_matrix_avg,cost_matrix_std,cost_matrix_skew,stdTime,mst_length,mst_length_avg,mst_length_std,mst_length_skew,mst_degree_avg,mst_degree_std,mst_degree_skew,mstTime,cluster_distance_avg,cluster_distance_std,cluster_distance_skew,clusterTime,tour_const_heu_avg,tour_const_std,tour_const_skew,ls_impov_per_step_avg,ls_impov_per_step_std,ls_impov_per_step_skew,ls_steps_2lm_avg,ls_steps_2lm_std,ls_steps_2lm_skew,ls_maxdist_avg,ls_maxdist_std,ls_maxdist_skew,ls_bestsol_avg,ls_bestsol_std,ls_bestsol_skew,ls_backbone_avg,ls_backbone_std,ls_backbone_skew,lpTime,bc_improv_per_cut_avg,bc_improv_per_cut_std,bc_improv_per_cut_skew,bc_upper_lower_ratio,bc_no1s_min,bc_no1s_q25,bc_no1s_q50,bc_no1s_q75,bc_no1s_max,bc_p1s,bc_pn1s,bcTime,acc,acfTime
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,0.001897,1.430042,1.491193,3.140406,0.927441,3.138509,0.621946,0.471871,0.487806,0.040004,0.369238,0.381506,0.706402,0.151182,0.666398,0.396278,23.0,0.024919,7.0,0.047305,5.0,0.096363,0.30250,0.45500,0.85500,0.739118,0.02250,0.458728,1.0,800.0,0.000002,0.0,818.771792,4.210875,0.0,0.533828,0.511093,1.350294,0.274833,1.350294,0.537736,4.0,1.0,5.0,13.24125,80.0,18.549748,79.0,1.400906,0.000219,0.008216,0.011043,0.339508,0.016342,0.339289,1.479878,0.000027,0.000219,0.005963,0.008050,0.070918,0.007696,0.070699,0.956124,800.0,510536.387909,0.536197,-38.723116,0.02,0.021553,0.021580,1.478438,0.079151,1.9975,0.341379,19.833117,0.02,0.363833,0.699597,14.795844,0.18,0.031919,0.000000e+00,0.000000e+00,0.002602,0.000824,-6.529526e+00,0.120875,0.006375,-1.050584,0.139375,2.478186e-02,1.798802e+00,20.848407,1.742264e-02,4.868199e+01,0.813835,0.279135,-0.884707,2.93,0.010508,4.320124,0.547672,0.010640,0.111111,0.500000,0.5,0.5,0.888889,0.89750,0.20625,0.22,380.733368,0.43
1,data/TSP/CEPS_benchmark/grid/04.tsp,0.001978,1.651404,1.641448,3.141593,0.934762,3.139615,0.569474,0.517514,0.488799,0.010197,0.405103,0.386497,0.717120,0.138593,0.706924,0.358587,1.0,0.384896,1.0,0.386497,1.0,0.386497,0.37375,0.66125,0.85125,0.984655,0.01875,0.510500,1.0,800.0,0.000002,0.0,840.219791,16.411909,0.0,0.516829,0.524481,1.381468,0.249095,1.381468,0.474936,2.0,1.0,5.0,13.65625,83.0,19.182819,82.0,1.404692,0.001058,0.024411,0.024615,0.067842,0.010731,0.066784,0.435948,0.000059,0.001058,0.018953,0.019898,0.067842,0.010276,0.066784,0.516433,800.0,524785.337480,0.473334,6.460961,0.02,0.046818,0.046877,0.435728,9.496167,1.9975,0.346841,6.204244,0.02,0.082693,0.098202,-1.880810,0.94,0.060078,0.000000e+00,0.000000e+00,0.005326,0.000000,-9.000000e-09,0.105000,0.000000,0.000000,0.150000,0.000000e+00,0.000000e+00,41.436466,6.030000e-07,-3.000000e-09,1.000000,0.000000,0.000000,0.91,0.001727,2.622374,0.410356,0.011900,0.250000,0.500000,0.5,0.5

In [3]:
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
t_c = 300
K = 2
n = 2
t_ini = train_instances.size * 10 * 10
t_i = 10
max_iter = 1

experiment = CepsMy1Experiment(
    t_c=t_c,
    t_ini=t_ini,
    t_i=t_i,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-14 23:10:57] INFO      [CEPS_MY_1] Start!
[2024-12-14 23:10:58] INFO      Initialization
[2024-12-14 23:10:58] INFO      Solver 1/2
[2024-12-14 23:10:58] INFO      Random config 1/10
[2024-12-14 23:10:58] DEBUG     =================================  Portfolio  ==================================
[2024-12-14 23:10:58] DEBUG     2220588739511820280 --> ASCENT_CANDIDATES=52;BACKBONE_TRIALS=1;BACKTRACKING=YES;CANDIDATE_SET_TYPE=ALPHA;EXTRA_CANDIDATES=7;EXTRA_CANDIDATE_SET_TYPE=QUADRANT;GAIN23=NO;GAIN_CRITERION=YES;INITIAL_STEP_SIZE=1;INITIAL_TOUR_ALGORITHM=QUICK-BORUVKA;INITIAL_TOUR_FRACTION=0.5820197920751;KICKS=3;KICK_TYPE=0;MAX_BREADTH=954863081;MAX_CANDIDATES=10;MOVE_TYPE=3;PATCHING_A=3;PATCHING_C=1;POPULATION_SIZE=8;RESTRICTED_SEARCH=YES;SUBGRADIENT=YES;SUBSEQUENT_MOVE_TYPE=6;SUBSEQUENT_PATCHING=NO
[2024-12-14 23:10:58] DEBUG     ================================================================================
[2024-12-14 23:10:58] DEBUG     executor start
[2024-12-14 23:11:06]

In [ ]:
# t_c = 60
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 2

# experiment = CepsExperiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 1

# experiment = CepsMy1Experiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [22]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [42]:
from src.database import db_connect
conn = db_connect()

In [15]:
import sqlite3
# conn = sqlite3.connect("_archive/7/MY_CEPS1.db")
conn = sqlite3.connect("database/2024_12_15_20_41_04.db")

In [16]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING


In [17]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id


In [18]:
df = pd.read_sql_query("SELECT * FROM results", conn)
df

,instance_id,solver_id,cost,time,comment,created_at


In [6]:
df["comment"].value_counts()

configuration     23457
validation         3243
initialization     2880
test1              1880
test2              1880
test3              1880
mutation            828
Name: comment, dtype: int64

In [7]:
_ = df.loc[df["comment"].str.contains("test")].pivot_table(index=["instance_id", "solver_id"], columns="comment", values="time").sort_values(["instance_id", "solver_id"])
_["range"] = _.max(axis=1) - _.min(axis=1)
_.sort_values(by="range")

comment                                                              test1  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/explosion/45.tsp        360611542263889118   10.00   
data/TSP/CEPS_benchmark/cluster_netgen/35.tsp   11967750618760666    10.00   
data/TSP/CEPS_benchmark/expansion/39.tsp        1813319980219163974  10.00   
                                                360611542263889118   10.00   
data/TSP/CEPS_benchmark/grid/31.tsp             2220723303743824310  10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/explosion/45.tsp        2220723303743824310   5.52   
data/TSP/CEPS_benchmark/cluster_netgen/41.tsp   11967750618760666     6.03   
data/TSP/CEPS_benchmark/linearprojection/10.tsp 1813319980219163974   8.37   
data/TSP/CEPS_benchmark/rotation/28.tsp         2220723303743824310   2.31   
                                                1813319980219163974   2.61   

comment                                                              test2  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/explosion/45.tsp        360611542263889118   10.00   
data/TSP/CEPS_benchmark/cluster_netgen/35.tsp   11967750618760666    10.00   
data/TSP/CEPS_benchmark/expansion/39.tsp        1813319980219163974  10.00   
                                                360611542263889118   10.00   
data/TSP/CEPS_benchmark/grid/31.tsp             2220723303743824310  10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/explosion/45.tsp        2220723303743824310   6.10   
data/TSP/CEPS_benchmark/cluster_netgen/41.tsp   11967750618760666     6.72   
data/TSP/CEPS_benchmark/linearprojection/10.tsp 1813319980219163974   8.52   
data/TSP/CEPS_benchmark/rotation/28.tsp         2220723303743824310   1.46   
                                                1813319980219163974   1.72   

comment                                                              test3  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/explosion/45.tsp        360611542263889118   10.00   
data/TSP/CEPS_benchmark/cluster_netgen/35.tsp   11967750618760666    10.00   
data/TSP/CEPS_benchmark/expansion/39.tsp        1813319980219163974  10.00   
                                                360611542263889118   10.00   
data/TSP/CEPS_benchmark/grid/31.tsp             2220723303743824310  10.00   
...                                                                    ...   
data/TSP/CEPS_benchmark/explosion/45.tsp        2220723303743824310   5.40   
data/TSP/CEPS_benchmark/cluster_netgen/41.tsp   11967750618760666     6.01   
data/TSP/CEPS_benchmark/linearprojection/10.tsp 1813319980219163974   9.11   
data/TSP/CEPS_benchmark/rotation/28.tsp         2220723303743824310   1.57   
                                                1813319980219163974   1.58   

comment                                                              range  
instance_id                                     solver_id                   
data/TSP/CEPS_benchmark/explosion/45.tsp        360611542263889118    0.00  
data/TSP/CEPS_benchmark/cluster_netgen/35.tsp   11967750618760666     0.00  
data/TSP/CEPS_benchmark/expansion/39.tsp        1813319980219163974   0.00  
                                                360611542263889118    0.00  
data/TSP/CEPS_benchmark/grid/31.tsp             2220723303743824310   0.00  
...                                                                    ...  
data/TSP/CEPS_benchmark/explosion/45.tsp        2220723303743824310   0.70  
data/TSP/CEPS_benchmark/cluster_netgen/41.tsp   11967750618760666     0.71  
data/TSP/CEPS_benchmark/linearprojection/10.tsp 1813319980219163974   0.74  
data/TSP/CEPS_benchmark/rotation/28.tsp         2220723303743824310   0.85  
                               

In [22]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
# _["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,11967750618760666,1813319980219163974,2220723303743824310,360611542263889118,best
instance_id,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,2.56,100.00,9.33,5.20,0
data/TSP/CEPS_benchmark/cluster/01.tsp,3.93,100.00,100.00,100.00,0
data/TSP/CEPS_benchmark/cluster/02.tsp,8.65,2.65,100.00,9.56,1
data/TSP/CEPS_benchmark/cluster/03.tsp,4.94,0.79,3.41,2.63,1
data/TSP/CEPS_benchmark/cluster/04.tsp,100.00,100.00,100.00,100.00,0
...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,0.64,0.48,2.60,0.85,1
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,0.36,0.52,1.29,1.41,0
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,0.55,0.71,2.22,1.08,0


In [23]:
_.iloc[:, :-1]

solver_id,11967750618760666,1813319980219163974,2220723303743824310,360611542263889118
instance_id,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,2.56,100.00,9.33,5.20
data/TSP/CEPS_benchmark/cluster/01.tsp,3.93,100.00,100.00,100.00
data/TSP/CEPS_benchmark/cluster/02.tsp,8.65,2.65,100.00,9.56
data/TSP/CEPS_benchmark/cluster/03.tsp,4.94,0.79,3.41,2.63
data/TSP/CEPS_benchmark/cluster/04.tsp,100.00,100.00,100.00,100.00
...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,0.64,0.48,2.60,0.85
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,0.36,0.52,1.29,1.41
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,0.55,0.71,2.22,1.08


In [19]:
_["min"].describe()

count    470.000000
mean       3.247596
std       13.600657
min        0.080000
25%        0.572500
50%        0.900000
75%        1.687500
max      100.000000
Name: min, dtype: float64

In [40]:
(_["min"] > 90).value_counts()

False    466
True       4
Name: min, dtype: int64

In [41]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

3    160
0    129
1    123
2     54
Name: best, dtype: int64

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index